In [4]:
!.\myenv\Scripts\activate
!pip install opencv-python
!pip install pytesseract


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
# !sudo apt-get install tesseract-ocr # Linux

In [26]:
# Windows
!pip install pytesseract 

In [27]:
# Optional for viewing image
!pip install pillow

In [1]:
# Import required packages
import cv2
import pytesseract

# For image viewing:
from PIL import Image
from IPython.display import display

# Extras
import os
import shutil
import re



# vars
path_input = '../data/input/'
path_failed_items = '../data/filtered_fail/'
path_not_a_image = '../data/filtered_fail/not_an_image/'
path_filtered_in = '../data/filtered_in/'
path_filtered_out = '../data/filtered_out/'
path_filtered_log = '../data/filtered_log/'

os.makedirs(path_filtered_out, exist_ok=True)


In [2]:
# Mention the installed location of Tesseract-OCR in your system
#pytesseract.pytesseract.tesseract_cmd = '/bin/tesseract'
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Get the list of files in the directory (Excluding directories)
files = os.listdir(path_input)
files = [f for f in files if os.path.isfile(os.path.join(path_input, f))]
count = 0
for file_name in files:
    # Read image from which text needs to be extracted
    count += 1
    # Testing if image can be opened
    try:
        with Image.open(path_input + file_name) as img_look:
            pass
    except Exception as e:
        print(f"[ERROR] [NaI] Error processing file {file_name}: {e}")
        # Move the file to the filtered_out/not_an_image directory
        shutil.move(path_input + file_name, path_not_a_image + file_name)

        continue

    try:
        img = cv2.imread(path_input + file_name)

        # Preprocessing the image starts

        # Convert the image to gray scale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Performing OTSU threshold
        ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

        # Specify structure shape and kernel size.
        # Kernel size increases or decreases the area
        # of the rectangle to be detected.
        # A smaller value like (10, 10) will detect
        # each word instead of a sentence.
        rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))

        # Applying dilation on the threshold image
        dilation = cv2.dilate(thresh1, rect_kernel, iterations=1)

        # Finding contours
        contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        # Creating a copy of image
        im2 = img.copy()

        # A text file is created and flushed
        txt_file_name = file_name.replace(".jpg", ".txt").replace(".png", ".txt").replace(".jpeg", ".txt")
        with open(path_filtered_log + txt_file_name, "w") as file:
            file.write("")

        # Looping through the identified contours
        # Then rectangular part is cropped and passed on
        # to pytesseract for extracting text from it
        # Extracted text is then written into the text file
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)

            # Drawing a rectangle on copied image
            rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Cropping the text block for giving input to OCR
            cropped = im2[y:y + h, x:x + w]

            # Open the file in append mode
            with open(path_filtered_log + txt_file_name, "a") as file:
                # Apply OCR on the cropped image
                text = pytesseract.image_to_string(cropped)

                # Appending the text into file
                file.write(text)
                file.write("\n")

        # Detect for specific text content
        with open(path_filtered_log + txt_file_name, "r") as file:
            content = file.read()

        pattern = r"Today \d{1,2}:\d{2} [AaPp][Mm]"
        if re.search(pattern, content):
            print(f"[MATCH] The pattern 'Today ##:## AM/PM' is present in the file: {file_name}.")
            
            # Wait for 5 seconds before moving the file
            # time.sleep(5)
            
            # Move the file using shutil.move()
            shutil.move(path_input + file_name, path_filtered_in + file_name)
        else:
            #print(f"The pattern 'Today ##:## AM/PM' is not present in the file: {file_name}.")
            shutil.move(path_input + file_name, path_filtered_out + file_name)
        
        if count % 10 == 0:
            print(f"[INFO] Processed {count} files.")
    except Exception as e:
        print(f"[ERROR] [?] Error processing file {file_name}: {e}")
        # Move the file to the filtered_out directory
        shutil.move(path_input + file_name, path_failed_items + file_name)

[MATCH] The pattern 'Today ##:## AM/PM' is present in the file: 0rja7kkon49c1.jpeg.
[INFO] Processed 10 files.
[MATCH] The pattern 'Today ##:## AM/PM' is present in the file: 0sm7llpz47zb1.jpg.
[INFO] Processed 20 files.
[INFO] Processed 30 files.
[INFO] Processed 40 files.
[MATCH] The pattern 'Today ##:## AM/PM' is present in the file: 0ypao7g4ytu71.jpg.
[INFO] Processed 50 files.
[MATCH] The pattern 'Today ##:## AM/PM' is present in the file: 12wgzzx1yv7c1.jpeg.
[INFO] Processed 60 files.
[MATCH] The pattern 'Today ##:## AM/PM' is present in the file: 1450a45o3qwb1.jpg.
[MATCH] The pattern 'Today ##:## AM/PM' is present in the file: 15anuldb7syb1.png.
[INFO] Processed 70 files.
[MATCH] The pattern 'Today ##:## AM/PM' is present in the file: 17h0sl3v2d3c1.jpeg.
[INFO] Processed 80 files.
[INFO] Processed 90 files.
[INFO] Processed 100 files.
[MATCH] The pattern 'Today ##:## AM/PM' is present in the file: 1ehp78odp57c1.jpeg.
[INFO] Processed 110 files.
[MATCH] The pattern 'Today ##:## 